In [ ]:
%pip install sign_language_translator

In [1]:
import json
import sign_language_translator as slt
from typing import Dict, List

path = slt.Assets.download("pk-dictionary-mapping.json", overwrite=False)[0]

with open(path) as f:
    dic: List[Dict[str, str | List[Dict[str, str | Dict[str, List[str]]]]]] = json.load(f)

tokens = {}
for collection in dic:
    # if collection["country"] != "pk":
    #     continue
    for mapping in collection["mapping"]:
        lang_to_text = mapping.get("token", {})
        for lang, txts in lang_to_text.items():
            for txt in txts:
                tokens.setdefault(lang, {})
                tokens[lang][txt]=0

{lang:len(tokens[lang]) for lang in tokens}

{'en': 1584, 'hi': 92, 'ur': 2072, 'latn-ur': 22}

In [13]:
import re
import requests
import urllib.parse

def get_google_search_results_count(query):
    query_ = urllib.parse.quote(query)
    url = f"https://www.google.com/search?as_epq={query_}"
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}

    try:
        response = requests.get(url, headers=headers)
        matches = re.findall(r"about ([0-9,]+) results", response.text.lower())
        if matches:
            return int(matches[0].replace(',', ''))
        else:
            print(f"Couldn't find the number of results for '{query}'")
            return 0
    except Exception as e:
        print(f"An error occurred ({query}):", e)
        return 0

In [ ]:
word_sense_regex = r"\([^\(\)]*\)"

def update_n_results(lang, word, data):
    if word in data[lang]:
        return
    n = get_google_search_results_count(word)
    if n:
        data[lang][word] = n

slt.utils.threaded_map(
    update_n_results,
    [(lang, re.sub(word_sense_regex, "", word), tokens)
        for lang in tokens
        for word in tokens[lang]
        if not tokens[lang][word]][1500:],
    time_delay=0.6,
    max_n_threads=500,
)

In [17]:
len({t:c for l,ts in tokens.items() for t,c in ts.items() if c>0})

67

In [19]:
with open("temp/label_counts.json", "w") as f:
    json.dump({
        lang: {t:c for t,c in ts.items() if c>0}
        for lang,ts in tokens.items()
    }, f, indent=2, ensure_ascii=False)